# Телеграм бот с применением RAG (Retrieval-Augmented Generation)

In [15]:
import os
from dotenv import load_dotenv
load_dotenv()

gigachat_token = os.getenv("GIGACHAT_TOKEN")
bot_token = os.getenv("BOT_TOKEN")

In [16]:
from langchain.vectorstores.chroma import Chroma
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.chat_models.gigachat import GigaChat
import warnings
warnings.filterwarnings('ignore')

GPT = GigaChat(
    credentials=gigachat_token,
    scope="GIGACHAT_API_PERS",
    model="GigaChat",
    streaming=False,
    verify_ssl_certs=False,
)

embeddings = HuggingFaceEmbeddings(
    model_name = "intfloat/multilingual-e5-large"
)

In [17]:
doc = open('data/history.txt',encoding='utf-8').read()
docs = doc.split(r"\\")
print(f"Всего фрагментов: {len(docs)}")
print(f"Макс длина фрагмента: {max(map(len,docs))}")

Всего фрагментов: 38
Макс длина фрагмента: 1128


In [18]:
res = embeddings.embed_documents(docs[0])
print(f"Длина эмбеддингов: {len(res[0])}")

Длина эмбеддингов: 1024


In [19]:
db = Chroma('vecstore',embeddings,'./db')
db.add_texts(docs)

['38f24426-8119-4846-a964-e1b979082162',
 '2ca496b4-5137-4b1e-8f24-9e7ac685484e',
 'a541d47e-e9b2-4591-b093-7f9d87028212',
 '88ba2881-c2aa-4d3e-a4ea-9ddf1af7e006',
 '75a5921c-945a-4004-bcd9-13b4b4bbb43c',
 'de0e9981-f247-43d1-8cc6-2f3adbcd7dfd',
 'bb52506f-4364-4fdc-8bbb-1ea15c903c2d',
 'ac1a299b-2ef4-4d93-af56-7a0493e463f8',
 'e37ae758-b179-419a-8e17-e9fd21bcc5e8',
 '6a08b998-0955-4d68-9fe3-8fd2d08866e2',
 '733c3743-1a08-42ec-82d6-042f04a23dcf',
 '05cc072d-701e-4a96-b343-741b2b8725ec',
 'b506fed3-11be-4fd8-8e79-6d1a807e78d2',
 'e02a3f39-fc4d-4f80-bca2-d8e0cb3bc88c',
 '0fe8f90c-0dcb-4e20-a88f-478adb91fa2d',
 '487f3e8a-6760-4ddc-9133-e2c4f4e4eed4',
 '0f36d3ef-9638-4c4a-ae19-06fafd8f6658',
 '160e39f7-b88d-4160-9fab-0a049587ef32',
 '6c287f9c-bf9c-432a-9b6a-23e43229b49e',
 '90f4d8e1-ddb4-48b5-9caa-b8a0ff30f804',
 '526114fb-9e2c-49bf-940c-fa9819d42359',
 '1dc3a2f8-78b7-4210-a77d-de312fa08336',
 '5f6b85c9-602d-4bdc-aa25-9834b91bc280',
 '9a0d2663-3ce8-4769-936e-af32c20e1532',
 'eb66899b-0866-

In [20]:
q = "сколько раз менялся император?"

retriever = db.as_retriever(search_kwargs={"k": 3 })
retriever.invoke(q)

[Document(metadata={}, page_content='\nКаковы были причины политической нестабильности после смерти Петра? Кто и каким образом занимал престол? Каково значение эпохи дворцовых переворотов в истории России?\n'),
 Document(metadata={}, page_content='\nКаковы были причины политической нестабильности после смерти Петра? Кто и каким образом занимал престол? Каково значение эпохи дворцовых переворотов в истории России?\n'),
 Document(metadata={}, page_content='\nПричины дворцовых переворотов\nПосле смерти Петра I Россия вступила в эпоху дворцовых переворотов (1725–1762), длившуюся до воцарения Екатерины II. За эти 37 лет на престоле сменилась череда правителей, многие из которых правили всего несколько лет. Впрочем, их власть зачастую была весьма условной: Пётр II стал императором и самодержцем всероссийским в 11 лет, а Иван VI — в два месяца. Были на престоле и невежественные люди, которые едва ли имели представление о том, как нужно управлять государством. В таких ситуациях у кормила власт

In [21]:
def answer(q):
  res = retriever.invoke(q)
  context = '\n'.join(x.page_content for x in res)
  prompt = f"""
  Прочитай следующий текст и используй его при ответе на вопрос далее.
  Используй только информацию из текста. Если в тексте нет ответа на данный вопрос,
  напиши, что не знаешь. Вот текст:\n{context}\nВопрос: {q}. Используй формат оформления Markdown для форматирования в telegram"""
  return GPT.invoke(prompt).content

answer(q)

'За период с момента смерти Петра I до восшествия на престол Екатерины II на российском троне сменилось **шесть** правителей.'

In [23]:
import telebot

bot = telebot.TeleBot(bot_token)

# Обработчик команды /start
@bot.message_handler(commands=['start'])
def start(message):
    # Отправляем приветственное сообщение
    bot.send_message(message.chat.id,
                     'Привет, я бот, который знает всё про дворцовые перевороты. Спрашивай!')

# Обработчик для всех входящих сообщений
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    ans = answer(message.text)
    bot.send_message(message.chat.id, ans, parse_mode='Markdown')

# Запуск бота
print("Бот готов к работе")
bot.polling(none_stop=True)

Бот готов к работе
